In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils
from sklearn.metrics import confusion_matrix, f1_score, auc, roc_curve, precision_recall_curve

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# Language Model

In [5]:
lm = True
clean = False

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/tmp_lm'),
 WindowsPath('../input/train.csv')]

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
allText_df = train_df.copy().append(test_df.copy(), sort=False)

In [7]:
if clean:
    train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
    test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x:clean_numbers(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

#train_x = train_df['question_text'].fillna('_##_').values
#test_x = test_df['question_text'].fillna('_##_').values

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [8]:
#print(df_train.head(3))

print(train_df.shape)
print(test_df.shape)

In [9]:
# df_train.head(3)

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = path/'train.csv',
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT)] 
    test = torchtext.data.TabularDataset( 
                path=path/'test.csv',
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


LABEL.batch_first


In [12]:
TEXT.build_vocab(train, test, vectors = "glove.6B.50d")

In [13]:
train_storage = train

In [252]:
#train = train_storage

In [253]:
train,_ = train.split(0.1)

In [254]:
from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size= 64, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)

test_iter = Iterator(
    test,
    batch_size = 64,
    sort = False,
    sort_within_batch = False,
    repeat = False)

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [255]:
#iter(train_iter).__next__().question_text
len(train_iter)
label_Size = next(iter(train_iter)).target
len(label_Size[label_Size==1])
#train_iter.batches
#train_iter = iter(train_iter)
#train_iter.__next__().question_text
#train_iter.__next__().target
#iter(train_iter).__next__().question_text.shape

2

In [256]:
#smaller,_ = train.split(0.1)

In [257]:
#smaller.examples[0].question_text

In [258]:
#len(train)
len(test_iter)

5872

In [259]:
#train.examples[0].question_text

In [260]:
TEXT.vocab.vectors.shape

torch.Size([527626, 50])

TEXT.vocab.freqs

In [312]:
embed_size = 50
hidden_size = 150
ver1 = False
sigBCE = True
bidir = True
layer_LSTM = 2
layer_Lin = 3
dropout_LSTM = 0.1
dropout_Lin = 0.1

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        
        hidden_size = 64
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size*4,2)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        #print(train.shape)
        h_embedding = self.embedding(train)
        #print(h_embedding.shape)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
        #print(h_embedding.shape)
        
        h_lstm, _ = self.lstm(h_embedding)
        #print(h_lstm.shape)
        h_gru, _ = self.gru(h_lstm)
        #print(h_gru.shape)
        
        avg_pool = torch.mean(h_gru,1)
        #print(avg_pool.shape)
        #avg_pool = avg_pool.view(-1,64)
        max_pool, _ = torch.max(h_gru,1)
        #print(max_pool.shape)
        
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        #print(conc.shape)
        #conc = self.dropout(conc)
        print(conc.shape)
        #out = nn.Sigmoid()(conc)
        
        out = nn.LogSoftmax(dim=1)(conc)
        
        #out = self.out(conc)
        #print(out[:,-1].shape)
        
        return out
    
    
    
    
batch = next(iter(train_iter)).question_text
net = NeuralNet()
#net(batch)

In [262]:
#batch = next(iter(train_iter)).question_text
#batch[30]
#layer_Lin

In [313]:
class LSTMTagger(nn.Module):
    def __init__(self):
        super(LSTMTagger, self).__init__()
                      
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        
        
        #self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm_Layer = nn.LSTM(embed_size, hidden_size,num_layers = layer_LSTM, dropout = dropout_LSTM, bidirectional = bidir)
        
        self.lin_Layer = []
        
        self.lin_Layer_Dropout = dropout_Lin
        
        if bidir:
            scale = 2
        else:
            scale = 1
        
        for _ in range(layer_Lin - 1):
            self.lin_Layer.append(nn.Linear(hidden_size*scale,hidden_size*scale))
            self.lin_Layer = nn.ModuleList(self.lin_Layer)
        if sigBCE:
            self.hidden2tag = nn.Linear(hidden_size*scale, 1)
        else:
            self.hidden2tag = nn.Linear(hidden_size*scale, 2)
    
    def forward(self, train):
        #print(train.shape)
        embeds = self.embedding(train)
        #print(embeds.shape)
        #print(embeds.view(len(train),64,-1).shape)
        #print(embeds.view(len(train),train.shape[1],-1).shape)
        lstm_out, _ = self.lstm_Layer(embeds)#embeds.view(len(train),train.shape[1],-1))
        #print(lstm_out.shape)
        
        if ver1:
            #print(lstm_out.shape)
            feature = lstm_out[-1,:,:] # -1 grabs last layer in list
            #print(feature.shape)
        
            for layer in self.lin_Layer:
                feature = layer(feature)
                feature = F.relu(feature)
                predict = self.hidden2tag(feature)
        
            feature = F.relu(feature)
            predict = self.hidden2tag(feature)
            if sigBCE:
                return nn.Sigmoid()(predict)
            else:
                return nn.LogSoftmax(dim=1)(predict)
        else:
           
            avg_pool = torch.mean(lstm_out,0)
            #print(avg_pool.shape)
            target_space = self.hidden2tag(avg_pool)#.view(len(train),-1))
            #print(target_space.shape)
            if sigBCE:
                return nn.Sigmoid()(target_space)
            else:
                return F.log_softmax(target_space, dim=1)
            #print(target_score.shape)

inputs_check = next(iter(train_iter)).question_text

#print(inputs_check.shape)

model = LSTMTagger()

with torch.no_grad():
    inputs = inputs_check
    #print(len(inputs))
    score = model(inputs)
    print(score.shape)
    print(score)

torch.Size([64, 1])
tensor([[0.5038],
        [0.5027],
        [0.5037],
        [0.5036],
        [0.5037],
        [0.5022],
        [0.5034],
        [0.5029],
        [0.5016],
        [0.5011],
        [0.5035],
        [0.5013],
        [0.5041],
        [0.5030],
        [0.5035],
        [0.5039],
        [0.5040],
        [0.5035],
        [0.5035],
        [0.5021],
        [0.5024],
        [0.5024],
        [0.5026],
        [0.5030],
        [0.5039],
        [0.5030],
        [0.5037],
        [0.5033],
        [0.5035],
        [0.5038],
        [0.5016],
        [0.5026],
        [0.5037],
        [0.5026],
        [0.5029],
        [0.5017],
        [0.5030],
        [0.5020],
        [0.5009],
        [0.5031],
        [0.5036],
        [0.5035],
        [0.5030],
        [0.5035],
        [0.5033],
        [0.5027],
        [0.5011],
        [0.5034],
        [0.5038],
        [0.5022],
        [0.5031],
        [0.5032],
        [0.5035],
        [0.5013],
        

In [314]:
def weight_init(m):
    if  isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

In [315]:
model = LSTMTagger()
model.apply(weight_init)
if sigBCE:
    loss_function = nn.BCELoss(reduction='none')
else:
    loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.2, momentum=0.9)

In [ ]:
validate = True

checking = next(iter(train_iter))
inputs_check = checking.question_text
target_check = checking.target


f1_train = []
auc_train = []
train_accu = []

avg_loss = 0

with torch.no_grad():
    inputs = inputs_check
    score1 = model(inputs)
    #print(score)
    
for epoch in range(10):
    model.train()
    avg_loss = 0
    training_set = iter(train_iter)
    model.zero_grad()
    all_preds = []
    all_actual = []
    for batch_num in range(len(train_iter)):
        
        sentence_data = next(training_set)
        sentence_in = sentence_data.question_text
        #target -> target.unsqueeze(1)
        if sigBCE:
            targets = sentence_data.target.float()#.unsqueeze(1)
        else:
            targets = torch.LongTensor(sentence_data.target)#.unsqueeze(1)
        optimizer.zero_grad()
        
        target_scores = model(sentence_in)
        
        loss = loss_function(target_scores.squeeze(1), targets)#.squeeze(1))#.reshape(64,1).squeeze(1))
        loss = (loss * (targets*14+1)/(targets*14+1).sum()).sum()#.mean()
        loss.backward()
        
        if sigBCE:
            prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
            accuracy = np.asarray(prediction == np.asarray(targets)).sum()/len(targets)*100
        else:
            prediction = target_scores.data.max(1)[1]
            accuracy = np.asarray(prediction==targets).sum()/len(targets)*100
        train_accu.append(accuracy)
        
        if sigBCE:
            all_preds.extend(target_scores.tolist())#torch.round(target_scores).tolist())
        else:
            all_preds.extend(torch.exp(target_scores).tolist())#torch.round(target_scores).tolist())
        all_actual.extend(targets.tolist())
        
        #print(np.asarray(target_scores)[:,1].max())
        
        #targets -> targets.reshape(64,1).squeeze(1)
        
        avg_loss += loss.item()
        
        #print(loss)
        if sigBCE:
            print(batch_num+1,"out of: ",len(train_iter), " With Loss of: ",loss.item()," Acc: ",accuracy, " Best Score of: ", np.asarray(target_scores).max())
        else:
            print(batch_num+1,"out of: ",len(train_iter), " With Loss of: ",loss.item()," Acc: ",accuracy, " Best Score of: ", np.asarray(torch.exp(target_scores))[:,1].max())
        
        optimizer.step()
    
    #model.eval()
    avg_loss = avg_loss/len(train_iter)
    
    #if validate:
        #avg_val
    
    
    #else:
    print("Training loss for epoch {} is {}".format(epoch + 1, avg_loss))
    
    
    #train_confusion_matrix = confusion_matrix(all_actual, all_preds).ravel()
    if sigBCE:
        pred_Clas = (np.asarray(all_preds) > 0.5).astype(int)#-2.65).astype(int)#
    else:
        pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)#-2.65).astype(int)#
    train_confusion_matrix = confusion_matrix(all_actual, pred_Clas).ravel()
    
    print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
    f1_train.append(f1_score(all_actual, pred_Clas))
    print("F1 score for epoch {} is {}".format(epoch + 1, f1_train[-1]))
    fpr, tpr, _ = roc_curve(all_actual, pred_Clas, pos_label=1)
    auc_train.append(auc(fpr, tpr))
    print("AUC for epoch {} is {}\n".format(epoch + 1, auc_train[-1]))

with torch.no_grad():
    inputs = inputs_check
    score2 = model(inputs)
    #print(score)
        
        

1 out of:  2041  With Loss of:  0.7078242897987366  Acc:  1.5625  Best Score of:  0.53529346
2 out of:  2041  With Loss of:  0.6793862581253052  Acc:  95.3125  Best Score of:  0.49622568
3 out of:  2041  With Loss of:  0.7397185564041138  Acc:  90.625  Best Score of:  0.45170102
4 out of:  2041  With Loss of:  0.6710286140441895  Acc:  95.3125  Best Score of:  0.45772377
5 out of:  2041  With Loss of:  0.7044904232025146  Acc:  90.625  Best Score of:  0.46176317
6 out of:  2041  With Loss of:  0.7219658493995667  Acc:  90.625  Best Score of:  0.5096081
7 out of:  2041  With Loss of:  0.6641263365745544  Acc:  73.4375  Best Score of:  0.5970915
8 out of:  2041  With Loss of:  0.6130262613296509  Acc:  84.375  Best Score of:  0.65311146
9 out of:  2041  With Loss of:  0.6637488007545471  Acc:  90.625  Best Score of:  0.6803183
10 out of:  2041  With Loss of:  0.6823036670684814  Acc:  84.375  Best Score of:  0.71590024
11 out of:  2041  With Loss of:  0.7127294540405273  Acc:  70.3125  B

88 out of:  2041  With Loss of:  0.5991429686546326  Acc:  67.1875  Best Score of:  0.83838075
89 out of:  2041  With Loss of:  0.6968387365341187  Acc:  62.5  Best Score of:  0.86916745
90 out of:  2041  With Loss of:  0.6687873601913452  Acc:  50.0  Best Score of:  0.8883598
91 out of:  2041  With Loss of:  0.6300121545791626  Acc:  48.4375  Best Score of:  0.87979513
92 out of:  2041  With Loss of:  0.5952639579772949  Acc:  48.4375  Best Score of:  0.87692165
93 out of:  2041  With Loss of:  0.6341680884361267  Acc:  59.375  Best Score of:  0.86297137
94 out of:  2041  With Loss of:  0.6974425315856934  Acc:  73.4375  Best Score of:  0.89352936
95 out of:  2041  With Loss of:  0.6679660081863403  Acc:  57.8125  Best Score of:  0.8825971
96 out of:  2041  With Loss of:  0.5764116048812866  Acc:  53.125  Best Score of:  0.9046061
97 out of:  2041  With Loss of:  0.6145943403244019  Acc:  79.6875  Best Score of:  0.85516363
98 out of:  2041  With Loss of:  0.5814437866210938  Acc:  82

174 out of:  2041  With Loss of:  0.739393413066864  Acc:  56.25  Best Score of:  0.86976063
175 out of:  2041  With Loss of:  0.4852377772331238  Acc:  79.6875  Best Score of:  0.95925146
176 out of:  2041  With Loss of:  0.3704402446746826  Acc:  93.75  Best Score of:  0.90120935
177 out of:  2041  With Loss of:  0.44643446803092957  Acc:  95.3125  Best Score of:  0.8994443
178 out of:  2041  With Loss of:  0.5330154299736023  Acc:  90.625  Best Score of:  0.8463084
179 out of:  2041  With Loss of:  0.5425804853439331  Acc:  90.625  Best Score of:  0.9729417
180 out of:  2041  With Loss of:  0.7584200501441956  Acc:  76.5625  Best Score of:  0.9990049
181 out of:  2041  With Loss of:  0.6397473216056824  Acc:  70.3125  Best Score of:  0.9765286
182 out of:  2041  With Loss of:  0.7980847954750061  Acc:  54.6875  Best Score of:  0.9990798
183 out of:  2041  With Loss of:  0.7678173184394836  Acc:  51.5625  Best Score of:  0.9983524
184 out of:  2041  With Loss of:  0.758500337600708  

260 out of:  2041  With Loss of:  0.4994264841079712  Acc:  84.375  Best Score of:  0.9821862
261 out of:  2041  With Loss of:  0.629545271396637  Acc:  73.4375  Best Score of:  0.99884987
262 out of:  2041  With Loss of:  0.592332124710083  Acc:  64.0625  Best Score of:  0.9997265
263 out of:  2041  With Loss of:  0.4294728636741638  Acc:  68.75  Best Score of:  0.99533963
264 out of:  2041  With Loss of:  0.4726932644844055  Acc:  64.0625  Best Score of:  0.99976665
265 out of:  2041  With Loss of:  0.7706127166748047  Acc:  67.1875  Best Score of:  0.9888636
266 out of:  2041  With Loss of:  0.5069707036018372  Acc:  78.125  Best Score of:  0.97740763
267 out of:  2041  With Loss of:  0.6662675738334656  Acc:  92.1875  Best Score of:  0.65820074
268 out of:  2041  With Loss of:  0.6456685066223145  Acc:  95.3125  Best Score of:  0.6947252
269 out of:  2041  With Loss of:  0.4146828055381775  Acc:  98.4375  Best Score of:  0.77422595
270 out of:  2041  With Loss of:  0.47748354077339

346 out of:  2041  With Loss of:  0.4532528519630432  Acc:  84.375  Best Score of:  0.9423546
347 out of:  2041  With Loss of:  0.297214537858963  Acc:  92.1875  Best Score of:  0.9839901
348 out of:  2041  With Loss of:  0.44197866320610046  Acc:  79.6875  Best Score of:  0.94339585
349 out of:  2041  With Loss of:  0.42947643995285034  Acc:  84.375  Best Score of:  0.9702804
350 out of:  2041  With Loss of:  0.6615301966667175  Acc:  90.625  Best Score of:  0.7933766
351 out of:  2041  With Loss of:  0.3158470094203949  Acc:  87.5  Best Score of:  0.9610365
352 out of:  2041  With Loss of:  0.35899680852890015  Acc:  90.625  Best Score of:  0.99977785
353 out of:  2041  With Loss of:  0.26283663511276245  Acc:  84.375  Best Score of:  0.9999367
354 out of:  2041  With Loss of:  0.4863319396972656  Acc:  85.9375  Best Score of:  0.9862629
355 out of:  2041  With Loss of:  0.45385533571243286  Acc:  82.8125  Best Score of:  0.9995271
356 out of:  2041  With Loss of:  0.3135332763195038

...

11 out of:  2041  With Loss of:  0.24315381050109863  Acc:  87.5  Best Score of:  0.9998348
12 out of:  2041  With Loss of:  0.39926448464393616  Acc:  79.6875  Best Score of:  0.99943084
13 out of:  2041  With Loss of:  0.5069283246994019  Acc:  76.5625  Best Score of:  0.9998381
14 out of:  2041  With Loss of:  0.35276275873184204  Acc:  85.9375  Best Score of:  0.9926288
15 out of:  2041  With Loss of:  0.2737160623073578  Acc:  82.8125  Best Score of:  0.99327564
16 out of:  2041  With Loss of:  0.3024265468120575  Acc:  79.6875  Best Score of:  0.9995122
17 out of:  2041  With Loss of:  0.17507390677928925  Acc:  79.6875  Best Score of:  0.9953296
18 out of:  2041  With Loss of:  0.2399241328239441  Acc:  84.375  Best Score of:  0.9843255
19 out of:  2041  With Loss of:  0.3961975872516632  Acc:  79.6875  Best Score of:  0.9512564
20 out of:  2041  With Loss of:  0.22731457650661469  Acc:  89.0625  Best Score of:  0.9998747
21 out of:  2041  With Loss of:  0.17879046499729156  Acc

98 out of:  2041  With Loss of:  0.12712615728378296  Acc:  87.5  Best Score of:  0.9969856
99 out of:  2041  With Loss of:  0.41222473978996277  Acc:  82.8125  Best Score of:  0.99899346
100 out of:  2041  With Loss of:  0.39856696128845215  Acc:  75.0  Best Score of:  0.9999105
101 out of:  2041  With Loss of:  0.3442368805408478  Acc:  78.125  Best Score of:  0.9981766
102 out of:  2041  With Loss of:  0.4341467320919037  Acc:  79.6875  Best Score of:  0.99947625
103 out of:  2041  With Loss of:  0.24171946942806244  Acc:  85.9375  Best Score of:  0.9941729
104 out of:  2041  With Loss of:  0.17775097489356995  Acc:  89.0625  Best Score of:  0.9984665
105 out of:  2041  With Loss of:  0.19581958651542664  Acc:  93.75  Best Score of:  0.93830407
106 out of:  2041  With Loss of:  0.3851107954978943  Acc:  81.25  Best Score of:  0.9994696
107 out of:  2041  With Loss of:  0.6744747757911682  Acc:  82.8125  Best Score of:  0.99966824
108 out of:  2041  With Loss of:  0.4106488823890686 

184 out of:  2041  With Loss of:  0.12518160045146942  Acc:  96.875  Best Score of:  0.9999732
185 out of:  2041  With Loss of:  0.4963747262954712  Acc:  92.1875  Best Score of:  0.9644752
186 out of:  2041  With Loss of:  0.1380346119403839  Acc:  93.75  Best Score of:  0.99182546
187 out of:  2041  With Loss of:  0.30329474806785583  Acc:  82.8125  Best Score of:  0.99549484
188 out of:  2041  With Loss of:  0.3780817985534668  Acc:  87.5  Best Score of:  0.9041681
189 out of:  2041  With Loss of:  0.22410370409488678  Acc:  85.9375  Best Score of:  0.99948454
190 out of:  2041  With Loss of:  0.3288422226905823  Acc:  89.0625  Best Score of:  0.9738418
191 out of:  2041  With Loss of:  0.2074832171201706  Acc:  85.9375  Best Score of:  0.99772567
192 out of:  2041  With Loss of:  0.35197776556015015  Acc:  79.6875  Best Score of:  0.9823513
193 out of:  2041  With Loss of:  0.6310203671455383  Acc:  75.0  Best Score of:  0.9896026
194 out of:  2041  With Loss of:  0.605669498443603

270 out of:  2041  With Loss of:  0.35940122604370117  Acc:  78.125  Best Score of:  0.97313154
271 out of:  2041  With Loss of:  0.6380226612091064  Acc:  84.375  Best Score of:  0.9645828
272 out of:  2041  With Loss of:  0.20908620953559875  Acc:  85.9375  Best Score of:  0.9737423
273 out of:  2041  With Loss of:  0.249215766787529  Acc:  85.9375  Best Score of:  0.9892953
274 out of:  2041  With Loss of:  0.2342827320098877  Acc:  90.625  Best Score of:  0.9520794
275 out of:  2041  With Loss of:  0.2445722371339798  Acc:  90.625  Best Score of:  0.89718
276 out of:  2041  With Loss of:  0.4457307755947113  Acc:  92.1875  Best Score of:  0.8067825
277 out of:  2041  With Loss of:  0.30107367038726807  Acc:  95.3125  Best Score of:  0.8528742
278 out of:  2041  With Loss of:  0.16507203876972198  Acc:  96.875  Best Score of:  0.9609308
279 out of:  2041  With Loss of:  0.5619428753852844  Acc:  95.3125  Best Score of:  0.7528132
280 out of:  2041  With Loss of:  0.4102001190185547 

356 out of:  2041  With Loss of:  0.4157043695449829  Acc:  68.75  Best Score of:  0.95711285
357 out of:  2041  With Loss of:  0.3116509020328522  Acc:  76.5625  Best Score of:  0.99712026
358 out of:  2041  With Loss of:  0.4636107087135315  Acc:  76.5625  Best Score of:  0.98894113
359 out of:  2041  With Loss of:  0.30011633038520813  Acc:  79.6875  Best Score of:  0.9976872
360 out of:  2041  With Loss of:  0.32029587030410767  Acc:  84.375  Best Score of:  0.95049256
361 out of:  2041  With Loss of:  0.2748080790042877  Acc:  84.375  Best Score of:  0.99984896
362 out of:  2041  With Loss of:  0.40186160802841187  Acc:  89.0625  Best Score of:  0.9580672
363 out of:  2041  With Loss of:  0.15066911280155182  Acc:  98.4375  Best Score of:  0.90687007
364 out of:  2041  With Loss of:  0.15456457436084747  Acc:  96.875  Best Score of:  0.9408621
365 out of:  2041  With Loss of:  0.2104537934064865  Acc:  98.4375  Best Score of:  0.9673827
366 out of:  2041  With Loss of:  0.02961754

In [ ]:
batch_size = 10
nb_classes = 2
target = torch.LongTensor(batch_size).random_(nb_classes)
#print((targets*5+1))
print((loss * (targets*5+1)).mean())

In [310]:
prediction = np.asarray(target_scores.squeeze(1)>0.5,dtype=float)
target_scores

tensor([[0.5147],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5152],
        [0.5148],
        [0.5145],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5148],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5147],
        [0.5148],
        [0.5147],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5146],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5145],
        [0.5148],
        [0.5147],
        [0.5147],
        [0.5147],
        [0.5147],
        [0.5148],
        [0.5146],
        [0.5147],
        [0.5146],
        [0.5147],
        [0.5147],
        [0

In [ ]:
#preddd = np.asarray(all_preds)[:,1]
#preddd_clas = (preddd>-0.45).astype(int)
#train_confusion_matrix = confusion_matrix(all_actual, preddd_clas).ravel()
#print("Confusion matrix for epoch {}, tn={}, fp={}, fn={}, tp={}".format(epoch + 1, *train_confusion_matrix))
#pred_Clas = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
#f1_score(all_actual, pred_Clas)
#pred_Clas

In [ ]:
preddd = (np.asarray(all_preds)[:,1] > np.asarray(all_preds)[:,0]).astype(int)
len(preddd[preddd>0])

In [ ]:
def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result

In [ ]:
def threshold_search_2(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [ ]:
#threshold_search_2(all_actual, preddd)
np.asarray(all_preds)[:,1].max()

In [ ]:
preddd = np.asarray(all_preds)[:,1]
preddd_clas = (preddd>-0.45).astype(int)
preddd.max()
len(preddd_clas[preddd_clas == 1])

In [ ]:
thr = threshold_search_2(all_actual, np.asarray(all_preds));thr

In [ ]:
thr = threshold_search(all_actual, np.asarray(all_preds), plot=True);thr

In [ ]:
thr['threshold']

def train_model(model, train_data, epochs, show=False):
    
    #optimizer = torch.optim.Adam(model.parameters())
    optimizer = optim.SGD(model.parameters(), lr=0.0001)
    #loss_function = nn.BCEWithLogitsLoss(reduction='mean')
    loss_function = nn.NLLLoss()
    
    print("The loss function being used is {}".format(loss_function))
    errors = []
    eval_errors = []
    f1_train = []
    auc_train = []
    f1_eval = []
    auc_eval = []
    
    num_training_batches = len(train_data)
    
    for epoch in range(epochs):
        print("Epoch {}".format(epoch + 1))
        print("Training mode")
        model.train()
        train_iter = iter(train_data)
        model.zero_grad()
        train_loss = 0
        all_preds = []
        all_actual = []
        for batch_num in range(num_training_batches):
            print("Batch {}".format(batch_num + 1))
            print("Out of {}".format(num_training_batches))
            batch = next(train_iter)
            text, class_vector = batch.question_text.transpose(0,1),batch.target.unsqueeze(1)
            #print(batch)
            #print(text.shape)
            #optimizer.zero_grad()
            text_pred = model(text)
            #print(text_pred)
            print(class_vector.shape)
            print(torch.max(class_vector,1)[0].shape)
            loss = loss_function(text_pred,class_vector.reshape(64,1).squeeze(1)) #torch.max(class_vector,1)[0])
            print(loss)
            print(text_pred.squeeze(1))
            print(torch.max(class_vector,1)[0])
            loss.backward()
            optimizer.step()
    

    
    

In [ ]:
#train_iters = iter(train_iter)

In [ ]:
#wee = next(train_iters)

In [ ]:
#wee.question_text.shape